In [1]:
# TSP：BerLin52
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]

In [2]:
# import 
import numpy as np
from random import sample
import random
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import pairwise_distances
import math
import pandas as pd

In [3]:
# 根据city_id坐标计算距离矩阵distance
distances = pairwise_distances(berlin52)
n_citys = len(berlin52)
# 每个city_id距离排序rank
dist_rank={}
for i in range(52):
    dist_rank[i] = sorted(distances[i])

In [4]:
# 初始解
def get_init_x(n_city = len(berlin52)):
    return np.random.permutation(n_city)
# 计算适应度函数：
def get_fitness(x):
    global distance
    
    fitness = sum([distances[x[i]][x[i+1]]  for i in range(len(x)-1)])
    fitness += distances[x[len(x)-1]][x[0]]
    return fitness

In [ ]:
# 一个city_id,前后相邻节点id，所对应的rank,计算rank_fit
# 对于所有的city_id,进行probabilistic_selection()
# 根据selected_id,找到相邻节点c1,c2

In [5]:
s_start = get_init_x()

In [6]:
s_start

array([15, 27, 29, 45, 49, 23, 17, 26, 50, 21, 30,  3, 18,  6, 44, 22,  9,
       42, 37, 19, 51, 38, 12, 36,  5, 40, 31, 46, 33,  7, 41, 28, 47, 20,
       16, 48,  2, 25, 34, 11, 13, 39, 35, 43, 10,  8,  0, 24,  1,  4, 32,
       14])

In [188]:
get_edges_for_city(s_start,5)

(3, 48)

In [205]:
def make_selection(components):
    sel = random.random()
    selected_id = 0
    for i in range(len(components)):
        sel -= components[i]
        if sel < 0 :
            selected_id = i
            
    return selected_id

def get_prob_selection(ordered_components,tao,exclude=[]):
    # 计算累计概率
    rank_prob = [ (i+1)**(-tao) for i in range(len(ordered_components))]
    rank_prob1 = [  i/ sum(rank_prob) for i in rank_prob]
    # 随机选择一个select_id
    selected_id = make_selection(rank_prob1)
    while selected_id in exclude:
        selected_id = make_selection(rank_prob1)
    return selected_id

def get_long_edge(city_id,edges,neighbors):
#     c = 0
    if distances[city_id][edges[0]] > distances[city_id][edges[1]]:
        c = edges[0]
    else:
        c = edges[1]
    return c



In [202]:

def vary_permutation(x,selected_id,new_neighbor,long_edge):
    new_x = []
    c1,c2 = x.index(selected_id),x.index(new_neighbor)
    p1,p2 = min(c1,c2),max(c1,c2)
    right = 0 if(c1==len(x)-1) else c1+1
    if right == long_edge:
        new_x = x[:p1+1]
        new_x += list(reversed(x[p1+1:p2]))
        new_x += x[p2:]
    else:
        new_x = x[:p1]
        new_x += list(reversed(x[p1:p2]))
        new_x += x[p2:]
    
    return new_x
        
def create_new_x(tao,x):
    city_fitnesses = get_rank_fitss(x)
    selected_city = get_prob_selection(list(reversed(city_fitnesses)),tao)
    edges = get_edges_for_city(x,selected_city)
    neighbors = dist_rank[selected_city]
    new_neighbor = get_prob_selection(neighbors,tao,edges)
    long_edge = get_long_edge(selected_city,edges, neighbors)
    return vary_permutation(x, selected_city, new_neighbor, long_edge)

In [183]:
def EO_search(max_iterations= 5,tao =1.2):
    sol = {}
    sol['x'] = list(get_init_x())
    sol['fitness'] = get_fitness(sol['x'])
    print(sol)
    for i in  range(max_iterations):
        new_x = create_new_x(tao,sol['x'])
        print(new_x)
        if get_fitness(sol['x']) > get_fitness(new_x):
            sol['x'] = new_x
            print(new_x)
    return sol

In [ ]:
# 分段检查
## 初始解 OK
## 目标函数fitness OK
## get_rank_fitss OK
## 

In [7]:
# get_edges_for_city(x,city_id)
def get_edges_for_city(x,city_id):
    c1,c2 = 0,0
    if x.index(city_id)== len(x)-1:
        c1,c2 = x[x.index(city_id)-1],x[0]
    else:
        c1,c2 = x[x.index(city_id)-1],x[x.index(city_id)+1]
    return c1,c2 
#  get_fitness_city(city_id,c1,c2)  
def get_neighbor_rank(city_id,c1,c2,ignore=[]):
    # rank_fit 约小距离越大
    r1 = dist_rank[city_id].index(distances[city_id][c1])
    r2 = dist_rank[city_id].index(distances[city_id][c2])
    rank_fit = 3/(r1+r2)
    return rank_fit
#  probabilistic_selection()
def get_rank_fitss(x):
    rank_fitss = []
    for city_id in x:
        c1,c2 = get_edges_for_city(x,city_id)
        rank_fitss.append(get_neighbor_rank(city_id,c1,c2))
    rank_fitss.sort()
    return rank_fitss